In [1]:
!pip install pyodbc -q

!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update
!sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   5921      0 --:--:-- --:--:-- --:--:--  5921
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    831      0 --:--:-- --:--:-- --:--:--   831
Hit:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubunt

In [2]:
import numpy as np
import pandas as pd
import pyodbc

import dataloader

In [3]:
from datetime import datetime

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

import helper

req = Request("http://insideairbnb.com/get-the-data/")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

# print(links)


In [4]:
def getdetailsfromurl(urltext):
    components = urltext.split('/')
    country = components[3]
    if len(components) == 9:
        province = components[4]
        city = components[5]
    else:
        province = components[3]
        city = components[3]
    return {'country': country, 'province':province, 'city': city}


# print(getdetailsfromurl('http://data.insideairbnb.com/new-zealand/2022-09-12/data/listings.csv.gz'))
# getdetailsfromurl('http://data.insideairbnb.com/united-states/dc/washington-dc/2022-09-14/data/listings.csv.gz')

In [5]:
data = {}
id = 0
for link in links:
        if link:
            if '.csv.gz' in link:
                data[id]= getdetailsfromurl(link)
                id+=1
                
data = pd.DataFrame(data).T
data['country'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['province'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)
data['city'].drop_duplicates().to_frame().reset_index().drop('index', axis =1)

,city
0,amsterdam
1,antwerp
2,asheville
3,athens
4,austin
...,...
110,winnipeg
111,zurich
112,ireland
113,malta


In [6]:
def getdatalinks(datasets = ['listings', 'calendar', 'reviews'], 
                 country = 'united-states', 
                 province = None, 
                 city = None):
    
    url_dict = {name:list() for name in datasets}
    for link in links:
        if link:
            if (not province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/' in link:
                        url_dict[dataset].append(link)
          
            if (province) and (not city):
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/' in link:
                        url_dict[dataset].append(link)

            if province and city:
                for dataset in datasets:
                    if f'/{dataset}.csv.gz' in link and f'/{country}/{province}/{city}/' in link:
                        url_dict[dataset].append(link)
    return(url_dict)

In [7]:
country = 'united-states'
province = 'nc'
city = 'asheville'
datasets = ['listings', 'calendar', 'reviews']

getdatalinks(datasets = datasets, country = country, province=province, city= city)

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz'],
 'calendar': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/calendar.csv.gz'],
 'reviews': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/reviews.csv.gz']}

In [8]:
urls = getdatalinks(datasets = datasets, 
                    country = country, 
                    # province=province, 
                    # city= city
                    )
urls

{'listings': ['http://data.insideairbnb.com/united-states/nc/asheville/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/austin/2022-09-12/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/ma/boston/2022-09-15/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/fl/broward-county/2022-09-21/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/ma/cambridge/2022-09-22/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/il/chicago/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/nv/clark-county-nv/2022-09-15/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/oh/columbus/2022-09-21/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/dallas/2022-09-14/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/co/denver/2022-09-26/data/listings.csv.gz',
  'http://data.insideairbnb.com/united-states/tx/fort-worth/20

In [21]:
listing_columns = ['id','country', 'province', 'city', 'listing_url', 
                #    'scrape_id', 
                   'last_scraped', 'source', 'name',
    #    'neighborhood_overview', 'picture_url', 
       'host_id',
    #    'host_url', 
       'host_name', 'host_since', 'host_location', 
       #'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 
    #    'host_thumbnail_url', 'host_picture_url',
    #    'host_neighbourhood', 
       'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 
    #    'neighbourhood',
    #    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 
       'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 
    #    'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

calendar_columns = ['listing_id', 
                    'date', 
                    'available', 
                    'price', 
                     #'adjusted_price',
                     # 'minimum_nights', 'maximum_nights'
                     ]

reviews_columns = ['listing_id', 
                   'date', 
                   'reviewer_id', 
                   'reviewer_name', 
                   'comments']

In [10]:
all_listings = pd.DataFrame()
all_calendar = pd.DataFrame()
all_reviews = pd.DataFrame()

for url in urls['listings']:
    # match = re.search('\d{4}-\d{2}-\d{2}', url)
    # print(datetime.strptime(match.group(), '%Y-%m-%d').date())
    #print(getdetailsfromurl(url))
    listings = (pd.read_csv(url)
            .assign(country = getdetailsfromurl(url)['country'],
                    province = getdetailsfromurl(url)['province'],
                    city = getdetailsfromurl(url)['city'])
    )
    all_listings = pd.concat([all_listings, listings])
all_listings = all_listings[listing_columns]
    
for url in urls['calendar']:
    calendar = pd.read_csv(url, parse_dates=['date'])
    calendar = calendar.loc[:,calendar_columns].loc[calendar['date'].isin(pd.date_range(start =datetime.now().date(), periods=10, freq='D'))]
    all_calendar = pd.concat([all_calendar, calendar])
    
for url in urls['reviews']:
    reviews =pd.read_csv(url, parse_dates=['date'])
    reviews = reviews.sort_values(by = 'date', ascending=False).groupby('listing_id').head(10)
    all_reviews = pd.concat([all_reviews, reviews])
all_reviews = all_reviews[reviews_columns]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (68) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
print(all_listings.shape)
print(all_calendar.shape)
print(all_reviews.shape)

(280043, 70)
(2799460, 4)
(1731906, 5)


# Write the tweaking functions and transform the datasets

In [ ]:
def cleanlistings(df):
    pass

def cleancalendar(df):
    pass

def cleanreviews(df):
    pass

In [34]:
all_listings['has_availability'].apply(lambda x: True if x ==  't' else False).value_counts()

True     264715
False     15328
Name: has_availability, dtype: int64

In [ ]:
all_listings['amenities'].str.replace('[', '').str.replace('"', '').str.replace(']', '')

In [ ]:
list_all = all_listings['amenities'].str.split(r'(?:,|;)\s*').dropna().to_numpy()

list_unique = np.unique(sum(list_all, []))

In [35]:
(
    all_listings.drop_duplicates(subset = ['id', 'listing_url'])
        .rename(columns = {'id':'listing_id'})
        .assign(listing_id = lambda x: x['listing_id'].astype('object'),
                host_id = lambda x: x['host_id'].astype('object'),
                last_scraped = lambda x: pd.to_datetime(x['last_scraped']).dt.normalize(),
                host_since = lambda x: pd.to_datetime(x['host_since']).dt.normalize(),
                calendar_last_scraped = lambda x: pd.to_datetime(x['calendar_last_scraped']).dt.normalize(),
                first_review = lambda x: pd.to_datetime(x['first_review']).dt.normalize(),
                last_review = lambda x: pd.to_datetime(x['last_review']).dt.normalize(),
                price = lambda x: x['price'].str.replace('$', '').str.replace(',', '').astype('float'),
                has_availability = lambda x: x['has_availability'].apply(lambda x: True if x ==  't' else False)
                    .astype('boolean'),
                instant_bookable = lambda x: x['instant_bookable'].apply(lambda x: True if x ==  't' else False)
                    .astype('boolean'),
                host_has_profile_pic = lambda x: x['host_has_profile_pic'].apply(lambda x: True if x ==  't' else False)
                    .astype('boolean'),
                host_identity_verified = lambda x: x['host_identity_verified'].apply(lambda x: True if x ==  't' else False)
                    .astype('boolean'),
                host_is_superhost = lambda x: x['host_is_superhost'].apply(lambda x: True if x ==  't' else False)
                    .astype('boolean'),
                
        ).info()
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 274790 entries, 0 to 6472
Data columns (total 58 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   listing_id                                    274790 non-null  object        
 1   country                                       274790 non-null  object        
 2   province                                      274790 non-null  object        
 3   city                                          274790 non-null  object        
 4   listing_url                                   274790 non-null  object        
 5   last_scraped                                  274790 non-null  datetime64[ns]
 6   source                                        274790 non-null  object        
 7   name                                          274773 non-null  object        
 8   host_id                                       274790 non

In [ ]:
all_listings.drop_duplicates(subset = ['id', 'listing_url'], inplace = True)

In [ ]:
all_listings = all_listings[listing_columns]

In [ ]:
all_listings=(
        all_listings.rename(columns = {'id':'listing_id'})
            .assign(listing_id = lambda x: x['listing_id'].astype('object'),
                    scrape_id = lambda x: x['scrape_id'].astype('object'),
                    host_id = lambda x: x['host_id'].astype('object'),
        )
    )

In [ ]:
all_calendar = all_calendar[calendar_columns]

In [ ]:


all_calendar=(
        all_calendar.assign(listing_id = lambda x: x['listing_id'].astype('object'),
        )
    )

In [ ]:
all_calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2752070 entries, 22 to 2361933
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   listing_id  object        
 1   date        datetime64[ns]
 2   available   object        
 3   price       object        
dtypes: datetime64[ns](1), object(3)
memory usage: 105.0+ MB


In [ ]:
dataloader.full_load(df = all_listings, tbl = "listings", hasindex = False)

DataFrame full loaded to Table: listings


In [ ]:
dataloader.full_load(df = all_calendar, tbl = "calendar", hasindex = False)

DataFrame full loaded to Table: calendar


In [ ]:
dataloader.full_load(df = all_reviews, tbl = "reviews", hasindex = False)